In [1]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization, Input, Dense, Dot, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import skipgrams
import numpy as np
import pandas as pd

train_path = os.path.join("train.csv")
test_path = os.path.join("test.csv")
df_train = pd.read_csv(train_path, encoding='ISO-8859-1', index_col="textID")
df_test = pd.read_csv(test_path, encoding='ISO-8859-1', index_col="textID")
df_train.head()

,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
textID,,,,,,,,,
cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


In [2]:
df_train.isna().sum()

,0
text,1
selected_text,1
sentiment,0
Time of Tweet,0
Age of User,0
Country,0
Population -2020,0
Land Area (Km²),0
Density (P/Km²),0


In [3]:
df_test.isna().sum()

,0
text,1281
sentiment,1281
Time of Tweet,1281
Age of User,1281
Country,1281
Population -2020,1281
Land Area (Km²),1281
Density (P/Km²),1281


In [4]:
df_train = df_train.dropna()
df_test = df_test.dropna()

In [5]:
df_train["text"].isna().sum()

np.int64(0)

In [6]:
from tensorflow.keras.layers import TextVectorization

vectorize_layer = TextVectorization(
    output_mode='int',
    standardize="lower_and_strip_punctuation",
    split="whitespace",
)

corpus = df_train["text"].values

vectorize_layer.adapt(corpus)

vectorized_train = vectorize_layer(corpus)
vectorized_train

<tf.Tensor: shape=(27480, 33), dtype=int64, numpy=
array([[  293,    17, 15185, ...,     0,     0,     0],
       [  413,   115,     2, ...,     0,     0,     0],
       [    6,  1335,    10, ...,     0,     0,     0],
       ...,
       [  225,    31,    12, ...,     0,     0,     0],
       [   20,     9,    28, ...,     0,     0,     0],
       [   29,    30,  6480, ...,     0,     0,     0]])>

In [7]:
vectorize_layer.get_vocabulary()[:10]

['',
 '[UNK]',
 np.str_('i'),
 np.str_('to'),
 np.str_('the'),
 np.str_('a'),
 np.str_('my'),
 np.str_('and'),
 np.str_('you'),
 np.str_('it')]

In [8]:
window_size = 2  # Puedes ajustar el tamaño de ventana

def generate_skipgrams_py(sequence):
    # sequence: tf.Tensor
    sequence = sequence.numpy()
    pairs, labels = skipgrams(
        sequence,
        vocabulary_size=vocab_size,
        window_size=window_size,
        negative_samples=2  # Ajusta según necesidad
    )
    if not pairs:
        # Si no hay pares, devuelve arrays vacíos
        return (np.array([], dtype=np.int32), np.array([], dtype=np.int32), np.array([], dtype=np.float32))
    targets, contexts = zip(*pairs)
    return (np.array(targets, dtype=np.int32),
            np.array(contexts, dtype=np.int32),
            np.array(labels, dtype=np.float32))

def tf_generate_skipgrams(sequence):
    targets, contexts, labels = tf.py_function(
        generate_skipgrams_py,
        inp=[sequence],
        Tout=[tf.int32, tf.int32, tf.float32]
    )

    targets.set_shape([None])
    contexts.set_shape([None])
    labels.set_shape([None])
    return tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))

In [9]:
# Construye el dataset plano de (target, context), label
vocab_size = len(vectorize_layer.get_vocabulary())
train_ds = tf.data.Dataset.from_tensor_slices(vectorized_train)
train_ds = train_ds.flat_map(tf_generate_skipgrams)
train_ds = train_ds.shuffle(10000)
train_ds = train_ds.map(
    lambda x, y: (
        (tf.one_hot(x[0], vocab_size), tf.one_hot(x[1], vocab_size)),
        y
    ),
    num_parallel_calls=tf.data.AUTOTUNE
)
train_ds = train_ds.batch(128).prefetch(tf.data.AUTOTUNE)

In [ ]:
embedding_dim = 64

input_target = Input(shape=(vocab_size,), name='target')
input_context = Input(shape=(vocab_size,), name='context')

shared_embedding = Dense(embedding_dim, use_bias=False, name='shared_embedding')

target_embedding = shared_embedding(input_target)
context_embedding = shared_embedding(input_context)

dot_product = Dot(axes=1)([target_embedding, context_embedding])
output = Activation('sigmoid')(dot_product)

model = Model(inputs=[input_target, input_context], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

model.fit(train_ds, epochs=10)

weights = model.get_layer('shared_embedding').get_weights()[0]
print("Matriz de embeddings shape:", weights.shape)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ target (InputLayer) │ (None, 29164)     │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ context             │ (None, 29164)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_embedding    │ (None, 64)        │  1,866,496 │ target[0][0],     │
│ (Dense)             │                   │            │ context[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot (Dot)           │ (None, 1)         │          0 │ shared_embedding… │
│                     │                   │            │ shared_embedding… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 1)         │          0 │ dot[0][0]         │
│ (Activation)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,866,496 (7.12 MB)

 Trainable params: 1,866,496 (7.12 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
28818/28818 ━━━━━━━━━━━━━━━━━━━━ 363s 13ms/step - accuracy: 0.7975 - loss: 0.4701
Epoch 2/10
    5/28818 ━━━━━━━━━━━━━━━━━━━━ 6:14 13ms/step - accuracy: 0.7559 - loss: 0.7718    

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


28818/28818 ━━━━━━━━━━━━━━━━━━━━ 352s 12ms/step - accuracy: 0.8116 - loss: 0.5283
Epoch 3/10
26886/28818 ━━━━━━━━━━━━━━━━━━━━ 23s 12ms/step - accuracy: 0.8184 - loss: 0.5077